In [ ]:
WITH eth_price AS (
  SELECT
    DATE_TRUNC('day', a.hour) AS day,
    AVG(a.price) AS price_usd
  FROM ethereum.core.fact_hourly_token_prices a
  WHERE a.symbol = 'WETH'
  GROUP BY 1
),
transfers AS (
  SELECT
    DATE_TRUNC('day', t.block_timestamp) AS date,
    t.origin_from_address,
    t.amount_usd,
    ft.tx_fee * ep.price_usd AS fee_usd
  FROM ethereum.core.ez_token_transfers t
  JOIN ethereum.core.fact_transactions ft ON t.tx_hash = ft.tx_hash
  JOIN ethereum.core.dim_labels l_from ON t.from_address = l_from.address
  JOIN eth_price ep ON DATE_TRUNC('day', t.block_timestamp) = ep.day
  WHERE t.symbol = 'WETH'
    AND t.amount_usd IS NOT NULL
    AND t.amount_usd > 0
    AND ft.status = 'SUCCESS'
    AND l_from.label_type = 'dex'
    AND t.block_timestamp >= '2022-10-11' AND t.block_timestamp <= '2022-12-12'
),
transfer_summary AS (
  SELECT
    date,
    origin_from_address,
    SUM(CASE WHEN amount_usd > 0 THEN amount_usd ELSE 0 END) AS inflow,
    SUM(CASE WHEN amount_usd < 0 THEN amount_usd ELSE 0 END) AS outflow,
    SUM(fee_usd) AS total_fees,
    COUNT(*) AS total_transactions
  FROM transfers
  GROUP BY date, origin_from_address
),
aggregated_summary AS (
  SELECT
    date,
    COUNT(DISTINCT date) AS total_dates,
    SUM(inflow) AS total_inflow,
    SUM(outflow) AS total_outflow,
    SUM(total_fees) AS total_fees,
    SUM(total_transactions) AS total_transactions
  FROM transfer_summary
  GROUP BY date
)
SELECT
  date,
  total_inflow + total_outflow AS total_volume,
  total_fees,
  total_transactions
FROM aggregated_summary
ORDER BY date;
